In [23]:
from PIL import Image
import os, glob

import tensorflow as tf
from tensorflow import keras
import numpy as np

import matplotlib.pyplot as plt

In [27]:
def load_data(img_path,source):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print(source,"의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/workplace/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path,"train")
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

image_dir_path_1 = os.getenv("HOME") + "/workplace/rock_scissor_paper_1"
(x_test, y_test)=load_data(image_dir_path_1,"test")
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화


train 의 이미지 개수는 300 입니다.
test 의 이미지 개수는 300 입니다.


In [47]:
n_channel_1=256
n_channel_2=512
n_dense=64
n_train_epoch=6

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [48]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 3s 324ms/step - loss: 57.6112 - accuracy: 0.2600
Epoch 2/10
10/10 [==============================] - 0s 10ms/step - loss: 1.7577 - accuracy: 0.3267
Epoch 3/10
10/10 [==============================] - 0s 8ms/step - loss: 1.3233 - accuracy: 0.3567
Epoch 4/10
10/10 [==============================] - 0s 8ms/step - loss: 0.8784 - accuracy: 0.6467
Epoch 5/10
10/10 [==============================] - 0s 7ms/step - loss: 0.5789 - accuracy: 0.7967
Epoch 6/10
10/10 [==============================] - 0s 7ms/step - loss: 0.2781 - accuracy: 0.9500
Epoch 7/10
10/10 [==============================] - 0s 7ms/step - loss: 0.1311 - accuracy: 0.9833
Epoch 8/10
10/10 [==============================] - 0s 7ms/step - loss: 0.2203 - accuracy: 0.9000
Epoch 9/10
10/10 [==============================] - 0s 7ms/step - loss: 0.1005 - accuracy: 0.9833
Epoch 10/10
10/10 [==============================] - 0s 7ms/step - loss: 0.0541 - accuracy: 0.9833


In [49]:
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 3.2509 - accuracy: 0.3767
test_loss: 3.2508749961853027 
test_accuracy: 0.3766666650772095
